In [1]:
from flask import Flask, request, jsonify, render_template, send_file, Response
from ultralytics import YOLO
import cv2
import numpy as np
import os
from werkzeug.utils import secure_filename
from threading import Thread, Lock
import tempfile
import time

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = tempfile.mkdtemp()
app.config['MAX_CONTENT_LENGTH'] = 100 * 1024 * 1024  # 100MB limit

# Load YOLO model
model = YOLO("best.pt")  

# Global variables for camera stream
camera_frame = None
camera_lock = Lock()
camera_active = False

@app.route('/')
def home():
    return render_template('html_interface.html')

@app.route('/video_feed')
def video_feed():
    """Route для streaming видео с камеры с детекцией"""
    return Response(generate_frames(), 
                   mimetype='multipart/x-mixed-replace; boundary=frame')

def generate_frames():
    """Генератор кадров с детекцией для streaming"""
    global camera_frame, camera_active
    
    cap = cv2.VideoCapture(0)
    camera_active = True
    
    try:
        while camera_active:
            ret, frame = cap.read()
            if not ret:
                break
                
            # Детекция объектов
            results = model.predict(frame, conf=0.5)
            annotated_frame = results[0].plot()
            
            # Конвертируем в JPEG
            ret, buffer = cv2.imencode('.jpg', annotated_frame)
            frame = buffer.tobytes()
            
            with camera_lock:
                camera_frame = frame
            
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            
            time.sleep(0.03)  # ~30 FPS
            
    finally:
        cap.release()
        camera_active = False

@app.route('/start_camera')
def start_camera():
    """Запуск камеры"""
    global camera_active
    if not camera_active:
        Thread(target=generate_frames).start()
        return jsonify({'status': 'camera started'})
    return jsonify({'status': 'camera already running'})

@app.route('/stop_camera')
def stop_camera():
    """Остановка камеры"""
    global camera_active
    camera_active = False
    return jsonify({'status': 'camera stopped'})

@app.route('/get_latest_frame')
def get_latest_frame():
    """Получение последнего кадра"""
    with camera_lock:
        if camera_frame is not None:
            return Response(camera_frame, mimetype='image/jpeg')
    return '', 204

@app.route('/process_video', methods=['POST'])
def process_video():
    if 'video' not in request.files:
        return jsonify({'success': False, 'error': 'No video file uploaded'})
    
    video_file = request.files['video']
    if video_file.filename == '':
        return jsonify({'success': False, 'error': 'No selected file'})
    
    try:
        # Save original video
        filename = secure_filename(video_file.filename)
        input_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        video_file.save(input_path)
        
        # Create path for processed video
        output_filename = 'processed_' + filename
        output_path = os.path.join(app.config['UPLOAD_FOLDER'], output_filename)
        
        # Process video and measure time
        start_time = time.time()
        detected_signs, detection_count = process_video_file(input_path, output_path)
        processing_time = time.time() - start_time
        
        return jsonify({
            'success': True,
            'detected_signs': list(detected_signs),
            'detection_count': detection_count,
            'processing_time': processing_time,
            'output_filename': output_filename
        })
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        })

def process_video_file(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise Exception("Could not open video file")
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Create VideoWriter for saving results
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    detected_signs = set()
    detection_count = 0
    
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect signs
        results = model.predict(frame, conf=0.5)
        boxes = results[0].boxes
        names = model.names
        
        # Draw bounding boxes
        for box in boxes:
            cls_id = int(box.cls[0])
            label = names[cls_id]
            detected_signs.add(label)
            detection_count += 1
            
            # Draw rectangle and label
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        out.write(frame)
        frame_count += 1
    
    cap.release()
    out.release()
    
    return detected_signs, detection_count

@app.route('/download_video')
def download_video():
    filename = request.args.get('filename')
    if not filename:
        return "Filename not provided", 400
    
    filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    if not os.path.exists(filepath):
        return "File not found", 404
    
    return send_file(filepath, as_attachment=True)

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

def start_server():
    try:
        templates_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)), 'templates')
    except NameError:
        templates_dir = os.path.join(os.getcwd(), 'templates')
    
    template_path = os.path.join(templates_dir, 'html_interface.html')
    
    if not os.path.exists(template_path):
        raise FileNotFoundError(f"Template file not found at {template_path}. Please create it.")
    
    print("Starting Flask server...")
    Thread(target=run_flask).start()
    time.sleep(1)
    print(f"Server should be running at http://localhost:5000")

start_server()

Starting Flask server...
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.19.249:5000
Press CTRL+C to quit


Server should be running at http://localhost:5000
